In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import time
print(tf.__version__)

1.10.0


In [2]:
import mnistdata
mnist = mnistdata.read_data_sets("data/mnist", one_hot=True, reshape=False)

Successfully unzipped train-images-idx3-ubyte.gz
Successfully unzipped train-labels-idx1-ubyte.gz
Successfully unzipped t10k-images-idx3-ubyte.gz
Successfully unzipped t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.images.shape)
print(mnist.test.labels.shape)

(60000, 28, 28, 1)
(10000, 10)


In [4]:
learning_rate = 1e-4
batch_size = 100
display_step = 1000
test_size = mnist.test.labels.shape[0]

n_input = 28
n_steps=  28
n_hidden = 64
n_classes = 10

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_input])
Y_ = tf.placeholder(tf.float32, [None, n_classes])

W = tf.Variable(tf.truncated_normal([n_hidden, n_classes], stddev=0.1))
b = tf.Variable(tf.truncated_normal([n_classes], stddev=0.1))

In [6]:
def get_RNN(x, w, b, mode):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps)
 
    if mode == 1:
        # single lstm layer
        cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
        out, state = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
        pred = tf.nn.softmax(tf.matmul(out[-1], w) + b)
 
    if mode == 2:
        # multilayer rnn (2 layers)
        n_layers = 2
        multi_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0) for _ in range(n_layers)], state_is_tuple=True)
        out, state = tf.nn.static_rnn(multi_cell, x, dtype=tf.float32)
        pred = tf.nn.softmax(tf.matmul(out[-1], w) + b)
 
    if mode == 3 :
        # bidirectional rnn
        w = tf.Variable(tf.truncated_normal([2*n_hidden, n_classes], stddev=0.1))
        fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
        bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
        out, _, _ = tf.contrib.rnn.static_bidirectional_rnn(fw_cell, bw_cell, x, dtype=tf.float32)
        pred = tf.nn.softmax(tf.matmul(out[-1], w) + b)
 
    return pred

In [7]:
Y1 = get_RNN(X, W, b, mode=1)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y1, labels=Y_))

correct_prediction = tf.equal(tf.argmax(Y1, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
# mnist
start = time.clock()
test_X = mnist.test.images
test_X = test_X.reshape([test_size, n_steps, n_input])
test_Y = mnist.test.labels
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 14 % test_accuracy = 9.85 %
step  1000, train_accuracy = 62 % test_accuracy = 54.83 %
step  2000, train_accuracy = 55 % test_accuracy = 70.46 %
step  3000, train_accuracy = 86 % test_accuracy = 79.45 %
step  4000, train_accuracy = 86 % test_accuracy = 81.89 %
step  5000, train_accuracy = 70 % test_accuracy = 83.24 %
step  6000, train_accuracy = 90 % test_accuracy = 83.83 %
step  7000, train_accuracy = 89 % test_accuracy = 84.88 %
step  8000, train_accuracy = 86 % test_accuracy = 90.16 %
step  9000, train_accuracy = 95 % test_accuracy = 92.4 %
training time:  118.1564773  s


In [9]:
Y2 = get_RNN(X, W, b, mode=2)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y2, labels=Y_))

correct_prediction = tf.equal(tf.argmax(Y2, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
# mnist
start = time.clock()
test_X = mnist.test.images
test_X = test_X.reshape([test_size, n_steps, n_input])
test_Y = mnist.test.labels
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 7 % test_accuracy = 9.82 %
step  1000, train_accuracy = 72 % test_accuracy = 78.57 %
step  2000, train_accuracy = 89 % test_accuracy = 87.53 %
step  3000, train_accuracy = 92 % test_accuracy = 91.12 %
step  4000, train_accuracy = 86 % test_accuracy = 91.68 %
step  5000, train_accuracy = 96 % test_accuracy = 93.08 %
step  6000, train_accuracy = 96 % test_accuracy = 93.62 %
step  7000, train_accuracy = 89 % test_accuracy = 94.31 %
step  8000, train_accuracy = 97 % test_accuracy = 94.31 %
step  9000, train_accuracy = 97 % test_accuracy = 95.1 %
training time:  224.3581229  s


In [11]:
Y3 = get_RNN(X, W, b, mode=3)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y3, labels=Y_))

correct_prediction = tf.equal(tf.argmax(Y3, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
# mnist
start = time.clock()
test_X = mnist.test.images
test_X = test_X.reshape([test_size, n_steps, n_input])
test_Y = mnist.test.labels
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 5 % test_accuracy = 9.11 %
step  1000, train_accuracy = 71 % test_accuracy = 62.11 %
step  2000, train_accuracy = 72 % test_accuracy = 69.07 %
step  3000, train_accuracy = 64 % test_accuracy = 71.41 %
step  4000, train_accuracy = 82 % test_accuracy = 80.35 %
step  5000, train_accuracy = 88 % test_accuracy = 87.94 %
step  6000, train_accuracy = 83 % test_accuracy = 89.56 %
step  7000, train_accuracy = 93 % test_accuracy = 90.78 %
step  8000, train_accuracy = 95 % test_accuracy = 91.61 %
step  9000, train_accuracy = 88 % test_accuracy = 91.99 %
training time:  124.3070677  s
